## Oblivious Transfer Algorithm

### Motivation

Let's imagine that we have two people: Paula and Valerie.

Paula knows a function $f: bool \rightarrow int$.

Valerie wants to query Paula for $f(true)$. The easiest way is for Valerie to simply ask for the value.

In [19]:
class Paula:
    def __init__(self):
        self.valerie = None
    
    def send_f(self, b):
        print(f"Paula recieves parameter {b}.")
        value = self._f(b)
        print(f"Paula sends value {value}.")
        return value
    
    def _f(self, b):
        if b:
            return 1000
        else:
            return 500

class Valerie:
    def __init__(self):
        self.paula = None

    def get_f(self):
        value = self.paula.send_f(True)
        print(f"Valerie obtains value {value}.")
    
class Protocol:
    def __init__(self):
        self.paula = Paula()
        self.valerie = Valerie()
        
        self.paula.valerie = self.valerie
        self.valerie.paula = self.paula

protocol = Protocol()
value = protocol.valerie.get_f()

Let's imagine now that Valerie doesn't want to tell Paula that she wants the value of $f$ for parameter $True$. She wants this transfer to be oblivious. The easiest way to do this is to ask for both of them, so Paula wouldn't know which value Valerie really wants.

In [21]:
class Paula:
    def __init__(self):
        self.valerie = None
    
    def send_f(self, b):
        print(f"Paula recieves parameter {b}.")
        value = self._f(b)
        print(f"Paula sends value {value}.")
        return value
    
    def _f(self, b):
        if b:
            return 1000
        else:
            return 500

class Valerie:
    def __init__(self):
        self.paula = None
        self.parameter = True

    def get_f(self):
        value_for_true = self.paula.send_f(True)
        value_for_false = self.paula.send_f(False)
        
        value = value_for_true if self.parameter else value_for_false
        print(f"Valerie obtains value {value}.")
    
class Protocol:
    def __init__(self):
        self.paula = Paula()
        self.valerie = Valerie()
        
        self.paula.valerie = self.valerie
        self.valerie.paula = self.paula

In [23]:
protocol = Protocol()
value = protocol.valerie.get_f()

Paula recieves parameter True.
Paula sends value 1000.
Paula recieves parameter False.
Paula sends value 500.
Valerie obtains value 1000.


This works okay for Valerie, whose privacy is respected, but what if Paula doesn't want to give too much information either?

Notice that in the first naive version Paula only gives one result to Valerie but in the second version she gives her both. This is not desirable.

We'll see a protocol that allows both parties to mantain privacy. That is:
- Paula only reveals one value to Valerie
- Valerie doesn't reveal which of the values she wants

We're going to need that both parties share a RSA key pair to be able to hide the requests.

The algorithm is as follows:

##### Idea
- Paula offers Valerie two random blinds, $x_{false}$ for $f(false)$ and $x_{true}$ for $f(true)$. 
- Valerie chooses the $x_b$ that corresponds to the bit $b$ she wants to know. She then generates her own random k and uses it to hide $x_b$ in a number $k'$.
- Paula receives the value $k'$ and does 2 calculations: one assuming that Valerie chose $x_{false}$ and the other one assuming she chose $x_{true}$. One of them makes sense and the other one doesn't, but Paula doesn't know which is which.
- So Paula sends both results. Valerie knows which value makes sense and which doesn't, so she obtains the result with one and discards the other.

##### Diagram

| Paula                                                                            |   Communication    | Valerie                                     |
| -------------------------------------------------------------------------------- |:-------------:| ----------------------------------------- |
|                                                                                  |               | Wants to know $f(b)$                  |
| Generates random blinds $x_{false}, x_{true}$                                                        | $\rightarrow$ | $x_{false}, x_{true}$                                |
|                                                                                  |               | Generates random noise $k$                        |
| $k'$                                                                              | $\leftarrow$  | Calculates $k' := (x_b + k^e) \text{ mod } N$ |
| Calculates: $k_{false} := (k' - x_{false})^d \text{ mod } N$ y $k_{true} := (k' - x_{true})^d \text{ mod } N$ |            |                                   ($k_b = k$, and $k_{!b}$ is noise)      |
| Calculates $v'_{false} := (v_{false} + k_{false}) \text{ mod } N$ y $v'_{true} := (v_{true} + k_{true}) \text{ mod } N$ |       $\rightarrow$        |                                      $v'_0, v'_1$     |
| | | Obtains $v_b = v'_b - k$ and discards $v_{!b}$ |

In [2]:
from Crypto.PublicKey import RSA
import random

def random_noise():
    # N is a really big number
    N = 2**2048
    return random.randrange(N)

class Paula:
    def __init__(self, key, modulus):
        self.valerie = None
        
        self.key = key
        self.modulus = modulus
        self.blinds = {
            True: random_noise(),
            False: random_noise(),
        }
    
    def send_blinds(self):
        print(f"\nPaula sends blinds {self.blinds}.")
        return self.blinds
    
    def calculate_k_b(self, bit, encrypted_key):
        base = (encrypted_key - self.blinds[bit] + self.modulus) % self.modulus
        return pow(base, self.key, self.modulus)
        
    def calculate_v_b(self, k_b, bit):
        return (k_b + self._f(bit)) % self.modulus
    
    def calculate_encrypted_value(self, bit, encrypted_key):
        k_b = self.calculate_k_b(bit, encrypted_key)
        return self.calculate_v_b(k_b, bit)
    
    def send_f(self, encrypted_key):
        print(f"\nPaula recieves parameter {encrypted_key}.")
    
        values = {
            b: self.calculate_encrypted_value(b, encrypted_key) 
            for b in [True, False]
        }
        print(f"\nPaula sends values {values}.")
        return values
    
    def _f(self, b):
        if b:
            return 1000
        else:
            return 500

class Valerie:
    def __init__(self, key, modulus):
        self.paula = None
        self.parameter = True
        self.key = key
        self.modulus = modulus
        
        self.k = random_noise()

    def get_correct_blind(self):
        return self.paula.send_blinds()[self.parameter]
    
    def get_correct_encrypted_value(self, hidden_key):
        return self.paula.send_f(hidden_key)[self.parameter]
    
    def hide_key(self, blind):
        return (blind + pow(self.k, self.key, self.modulus)) % self.modulus
    
    def retrieve_value(self, encrypted_value):
        return (encrypted_value - self.k + self.modulus) % self.modulus
    
    def get_f(self):
        blind = self.get_correct_blind()
        print(f"\nValerie calculates correct blind {blind}.")
        
        hidden_key = self.hide_key(blind)
        print(f"\nValerie calculates hidden key {hidden_key}.")
        
        encrypted_value = self.get_correct_encrypted_value(hidden_key)
        print(f"\nValerie receives hidden value {encrypted_value}.")
        
        value = self.retrieve_value(encrypted_value)
        print(f"\nValerie decrypts value {value}.")
    
class Protocol:
    def __init__(self):
        key_pair = RSA.generate(bits=2048)
        paula_key = key_pair.d
        valerie_key = key_pair.e
        modulus = key_pair.n
        
        self.paula = Paula(paula_key, modulus)
        self.valerie = Valerie(valerie_key, modulus)
        
        self.paula.valerie = self.valerie
        self.valerie.paula = self.paula
        
protocol = Protocol()
value = protocol.valerie.get_f()


Paula sends blinds {True: 11491655181835287411641733308903499505456163395213060576756313180758180729856651625917785993156064762874630368355681566108411360836640566097571806928765964322794986920662648970049105786309155516956797628962167889818672870310685279193514516811161801665919497651348006925935039883015526964012848270498018637290831077787519413335053467936598670801072060465969243674857487939426661057098934993260787029771384538884983027521850230403761339983200198564660407441888861365844033748084422710682211278500962196420397827500965665107559180136303108634808113681476480559004842411710478937067512168880506053533468260817635690103398, False: 10164009316430889412689475298544121337757967468147218909967918675596510426817131881257095497149717278023859773761165978229569629166687872695481063576774770383856509919013664094469108586368251759172961699232635244940473269793248321348195850005192001308783236721520400641054160353676257834107094414314005424555104036084852176969689066605889794737514

##### Explanation
Here we'll explain why this works.

Valerie calculates $k'$ as:

$k' := (x_b + k^e) \text{ mod } N$

When Paula calculates $k_b$, the following happens (we'll omit the modulo $N$ notation):

$k_b = (k' - x_b)^d = (x_b + k^e - x_b)^d = (k^e)^d = k$

This is true because $e$ and $d$ are a pair of RSA keys. On the other hand, for $k_{!b}$:

$k_{!b} = (k' - x_{!b})^d = (x_b + k^e - x_{!b})^d$

Which is a random value.

For Paula both values are random because she never saw $k$, only $k'$, so she can't recognise it.

This means that Paula sends to Valerie:
- $v'_b = v_b - k$
- $v'_{!b} = v_{!b} - (x_b + k^e - x_{!b})^d$

Valerie knows $k$ so she can obtain $v_b$. She can't calculate $(x_b + k^e - x_{!b})^d$ because she doesn't know $d$, so she can't decypher $v_{!b}$.